In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from keras.models import load_model

In [2]:
y = pd.read_csv(r"C:\Users\anant\Documents\GitHub\Taxonomic_classifier\Data\Taxonomic_data.csv", usecols=['Species'])
x7 = pd.read_csv(r"C:\Users\anant\Documents\GitHub\Taxonomic_classifier\Data\k(7)mer_data.csv").drop("Unnamed: 0", axis=1)
x6 = pd.read_csv(r"C:\Users\anant\Documents\GitHub\Taxonomic_classifier\Data\k(6)mer_data.csv").drop("Unnamed: 0", axis=1)
x5 = pd.read_csv(r"C:\Users\anant\Documents\GitHub\Taxonomic_classifier\Data\k(5)mer_data.csv").drop("Unnamed: 0", axis=1)


In [3]:
train_x7, test_x7, train_y7, test_y7 = train_test_split(x7, y, test_size=0.2, random_state=7)
train_x6, test_x6, train_y6, test_y6 = train_test_split(x6, y, test_size=0.2, random_state=7)
train_x5, test_x5, train_y5, test_y5 = train_test_split(x5, y, test_size=0.2, random_state=7)

In [4]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit(y)


LabelBinarizer()

In [5]:
k7_net = load_model("k7-net")
k6_net = load_model("k6-net")
k5_net = load_model("k5-net")

In [10]:
pred =  dict([[x,y] for x,y in zip(lb.inverse_transform(k6_net.predict(test_x6[10:15])), [4, 5, 6])])

In [11]:
pred

{'Vibrio parahaemolyticus': 4,
 'Enterococcus faecalis': 5,
 'Candidatus Hamiltonella defensa': 6}

In [22]:
i = 0
mismatch = 0
npredictions = 0
predictions = []
y_predicted = []
mismatch_species = []
mismatch_species_pred = []

for y in y.values:
    j=1
    a = k7_net.predict(test_x7[i:i+1])
    b = k6_net.predict(test_x6[i:i+1]) 
    c = k5_net.predict(test_x5[i:i+1])
    preds = [a, b, c]
    pred = dict([(x,y) for (x, y) in zip([max(x[0]) for x in preds],[str(lb.inverse_transform(x)[0]) for x in preds ])])
    b = [x for x in pred.values() if list(pred.values()).count(x)>1]
    sp = y
    
    if any(b):
        p_sp = b[0]
        npredictions += 1
        j = 2
        predictions.append(p_sp)
        y_predicted.append(sp)

    elif max(pred.keys())>0.99:
        p_sp = pred.get(max(pred.keys()))
        j=2
        npredictions += 1
        predictions.append(p_sp)
        y_predicted.append(sp)
    
    
    if p_sp!=sp and j==2 :
        mismatch_species.append(sp)
        mismatch_species_pred.append(p_sp)
        mismatch += 1
    
    i  += 1
print(mismatch)

298


In [42]:
i = 0
npredictions = 0
predictions = []

for y in x5.values[0:10]:
    j=1
    a = k7_net.predict(test_x7[i:i+1])
    b = k6_net.predict(test_x6[i:i+1]) 
    c = k5_net.predict(test_x5[i:i+1])
    preds = [a, b, c]
    pred = dict([(x,y) for (x, y) in zip([max(x[0]) for x in preds],
                [str(lb.inverse_transform(x)[0]) for x in preds ])])
                
    b = [x for x in pred.values() if list(pred.values()).count(x)>1]

    if any(b):
        p_sp = b[0]
        npredictions += 1
        predictions.append(p_sp)

    elif max(pred.keys())>0.99:
        p_sp = pred.get(max(pred.keys()))
        npredictions += 1
        predictions.append(p_sp)
    
    i  += 1
predictions

(['Lactobacillus reuteri',
  'Staphylococcus aureus',
  'Moraxella catarrhalis',
  'Bacillus anthracis',
  'Acinetobacter baumannii',
  'Listeria monocytogenes',
  'Micromonospora sp. L5',
  'Lactobacillus reuteri',
  'Shigella flexneri',
  'Staphylococcus epidermidis'],
 10)

In [35]:
from sklearn import metrics
f1 = metrics.f1_score(y_predicted, predictions, average = 'weighted')
f1

0.8663714580280101

In [38]:
a = pd.DataFrame(predictions)
a.value_counts()/sum(a.value_counts())*100



Escherichia coli                   7.917174
Salmonella enterica                3.613479
Streptococcus pneumoniae           2.598457
Staphylococcus aureus              2.233049
Bacillus cereus                    1.705238
                                     ...   
Ketogulonicigenium vulgare         0.040601
Jonquetella anthropi               0.040601
Janthinobacterium sp. Marseille    0.040601
Isosphaera pallida                 0.040601
gamma proteobacterium NOR5-3       0.040601
Length: 749, dtype: float64

In [14]:
i

2651